In [ ]:
import json
import numpy as np
from PIL import Image

# # Hàm tải trọng số từ các file .pth
# def load_weights(weights_dir):
#     weights = {}
#     import os
#     for weight_file in os.listdir(weights_dir):
#         layer_name = weight_file.replace('.pth', '')
#         weights[layer_name] = np.load(os.path.join(weights_dir, weight_file), allow_pickle=True)
#     return weights

# Hàm thực hiện convolution

def conv2d(input, filters, bias, stride=1, padding=1):
    c_out, c_in, kernel_size, _ = filters.shape
    h, w = input.shape[1], input.shape[2]
    
    # Tính kích thước đầu ra
    h_out = (h - kernel_size + 2 * padding) // stride + 1
    w_out = (w - kernel_size + 2 * padding) // stride + 1

    # Padding input
    input_padded = np.pad(input, ((0, 0), (padding, padding), (padding, padding)), mode='constant')
    output = np.zeros((c_out, h_out, w_out))

    # Tính convolution
    for out_c in range(c_out):
        for i in range(h_out):
            for j in range(w_out):
                region = input_padded[:, i*stride:i*stride+kernel_size, j*stride:j*stride+kernel_size]
                output[out_c, i, j] = np.sum(region * filters[out_c]) + bias[out_c]

    return output

# Hàm kích hoạt ReLU
def relu(x):
    return np.maximum(0, x)

# Hàm max pooling
def max_pool2d(input, pool_size=2, stride=2):
    c, h, w = input.shape
    h_out = (h - pool_size) // stride + 1
    w_out = (w - pool_size) // stride + 1
    output = np.zeros((c, h_out, w_out))

    for i in range(h_out):
        for j in range(w_out):
            region = input[:, i*stride:i*stride+pool_size, j*stride:j*stride+pool_size]
            output[:, i, j] = np.max(region, axis=(1, 2))
    return output

# Hàm flatten
def flatten(x):
    return x.reshape(-1)

# Hàm fully connected (Dense layer)
def dense(input, weights, bias):
    return np.dot(weights, input) + bias

# Tải cấu trúc và trọng số của mô hình
with open('model_structure.json', 'r') as f:
    model_structure = json.load(f)

import numpy as np
import os
import pickle

import numpy as np
import os

# Hàm tải trọng số mà không cần sử dụng pickle
def load_weights(weights_dir):
    weights = {}
    for weight_file in os.listdir(weights_dir):
        # Đọc trọng số từ tệp .npz hoặc các tệp numpy lưu trữ
        weight_data = np.load(os.path.join(weights_dir, weight_file), allow_pickle=True)
        
        for key in weight_data.keys():
            weights[key] = weight_data[key]  # Lưu trọng số trực tiếp

    return weights

weights = load_weights('model_weights')

# Danh sách các biển báo giao thông (label)
traffic_sign_labels = {
    0: 'Thang hoac Phai, cam queo Trai',
    1: 'Cam di nguoc chieu',
    2: 'Cam re trai',
    7: 'Canh bao co tre em',
    8: 'Cam dau xe',
    4: 'Di cham thoi',
    9: 'Cam dung va do xe',
    6: 'Huong di theo vach ke duong',
    3: 'Gap khuc phai',
    5: 'Huong phai di vung phai',
}

# Hàm chuẩn bị dữ liệu
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = image.resize((64, 64))
    image = np.asarray(image) / 255.0
    image = (image - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
    return image.transpose(2, 0, 1)  # Chuyển kênh thành (C, H, W)

# Hàm dự đoán
# Hàm dự đoán
def predict_traffic_sign(image_path):
    # Tiền xử lý ảnh
    x = preprocess_image(image_path)

    # Forward pass qua từng lớp
    x = conv2d(x, weights['conv1.weight/data.pkl'], weights['conv1.bias/data.pkl'])
    x = relu(x)
    x = max_pool2d(x)

    x = conv2d(x, weights['conv2.weight/data.pkl'], weights['conv2.bias/data.pkl'])
    x = relu(x)
    x = max_pool2d(x)

    x = flatten(x)

    x = dense(x, weights['fc1.weight/data.pkl'], weights['fc1.bias/data.pkl'])
    x = relu(x)

    x = dense(x, weights['fc2.weight/data.pkl'], weights['fc2.bias/data.pkl'])

    # Lấy nhãn dự đoán
    predicted_label_idx = np.argmax(x)
    return traffic_sign_labels.get(predicted_label_idx, "Unknown")


# Dự đoán
image_path = 'a.png'
predicted_label = predict_traffic_sign(image_path)
print(f'Predicted Traffic Sign: {predicted_label}')


UnpicklingError: A load persistent id instruction was encountered,
but no persistent_load function was specified.

In [3]:
weights = load_weights('model_weights')
print("Available keys in weights:", weights.keys())


Available keys in weights: dict_keys(['fc1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'conv1.weight', 'fc2.weight', 'conv1.bias', 'fc2.bias'])


In [4]:
for layer_name, params in weights.items():
    print(f"Layer: {layer_name}, Keys: {params.keys()}")


Layer: fc1.bias, Keys: dict_keys(['fc1.bias/data.pkl', 'fc1.bias/byteorder', 'fc1.bias/data/0', 'fc1.bias/version', 'fc1.bias/.data/serialization_id'])
Layer: conv2.weight, Keys: dict_keys(['conv2.weight/data.pkl', 'conv2.weight/byteorder', 'conv2.weight/data/0', 'conv2.weight/version', 'conv2.weight/.data/serialization_id'])
Layer: conv2.bias, Keys: dict_keys(['conv2.bias/data.pkl', 'conv2.bias/byteorder', 'conv2.bias/data/0', 'conv2.bias/version', 'conv2.bias/.data/serialization_id'])
Layer: fc1.weight, Keys: dict_keys(['fc1.weight/data.pkl', 'fc1.weight/byteorder', 'fc1.weight/data/0', 'fc1.weight/version', 'fc1.weight/.data/serialization_id'])
Layer: conv1.weight, Keys: dict_keys(['conv1.weight/data.pkl', 'conv1.weight/byteorder', 'conv1.weight/data/0', 'conv1.weight/version', 'conv1.weight/.data/serialization_id'])
Layer: fc2.weight, Keys: dict_keys(['fc2.weight/data.pkl', 'fc2.weight/byteorder', 'fc2.weight/data/0', 'fc2.weight/version', 'fc2.weight/.data/serialization_id'])
Laye